# Entropy, conditional entropy, and mutual information

Overview:
* In this tutorial, we will learn how to describe the information that is shared between two variables (mutual information). In other words, how much uncertainty reduction is there to be had about variable 1 by measuring variable 2? 

* These concepts were initially developed in communication theory to describe the efficacy of transmitting signals over a noisy medium (like a noisy telephone line). For example, suppose that we want to know how good a communication channel is, or its effeciency in reliably relaying a message from point A (a 'sender') to point B (a 'reciever').

* Basically this is just like asking, "we know how good the signal is at A, and we recieved the message at B - how much information about A is still in the received signal B?". 

* So that is the general gist of it, but right away you can see the potential applicability of this metric in many fields of neuroscience, psychology, engeneering, etc. In neuroscience, we're dealing with a series of communication channels that are corrupted by noise (e.g. synapses). It is therefore reasonable to ask: how much information from neuron A effectively propogates to neuron B? (or conversely, how much  information is lost?).

* However, this logic works for any combination of variables: two continuous variables, two discrete variables, one continuous and one discrete, etc. As a result, we can ask questions about any two variables really: how much information about median home  price is reflected in stock market fluctuations? etc.

* A few notes before we get started. First, we're going to be talking a lot about 'uncertainty' and 'uncertainty reduction'. While this is basically complementary to talking about certainty and an increase in certainty, we'll deal with the former terminology as it is embedded in some of the concepts that we'll discuss. 

* Second, we'll be dealing with variability in data, and how we can either attribute that variability in the data to 'noise' or to 'signals'. I.e. is the variability in one variable random wrt another variable? or does the variability in one variable systematically change with the variability in another? 

<div class="alert alert-info">
Finally, a lot of people think at this point, "why not just correlate the variables using a normal r-value?". There are a few answers to this, but the simplest is this: correlation assumes a linear relationship (or, in more complex forms, a known relationship or you have to assume a relationship) between variables. Mutual information does not, and can generally capture any form of linear or non-linear relationship between two variables. This makes it a very powerful and general purpose metric.
</div>

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# also define the default font we'll use for figures. 
fig_font = {'fontname':'Arial', 'size':'20'}

PART II: Entropy as a measure of variability

* Shannon Entropy is related to the variability of data, but is more specifically defined as the average uncertainty in a set of measurements 
* Consider coin tosses - lets say we have a balanced coin, and we flip it once. We can represent the outcome of a single toss as a 0 or a 1 (a head or a tail), and this has an entropy (uncertainty) of 1 bit. In other words, we would reduce our uncertainty completely after we observed the outcome, and that would correspond to a reduction of 1 bit. 
    * By extension, if we flipped the coin twice, then the entropy would be 2 bits (00, 01, 10, or 11) because we would reduce our uncertainty by 2 bits after we observed the outcome. 

* To keep with the coin analogy...when the coin is balanced and heads and tails are equally likely, then the entropy is highest because uncertainty is maximized and flipping the coin will give you 1 bit of information.  
* To see why entropy is maximized in this situation, consider a biased coin  that comes up heads 60% the time. 
    * In this case, we could predict the outcome of the coin flip better than chance simply by going with our prior of 'heads'. Thus, the entropy assoicated with a biased coin is less than the entropy associated with the unbiased coin, because the reduction in uncertainty is lower with the biased coin than with the unbaised coin. 
    * We learn less after flipping the biased coin than we do when we flip the unbiased coin. 
* This example brings up an important point: entropy as a measure of uncertainty is maximized when all possible outcomes are equally likely because you have no prior info upon which to make an educated guess about the outcome (i.e. a uniform distribution on the span of possible outcomes has the highest entropy). 

## To figure out the entropy of a measurement in terms of bits (the most common metric), we can use the log2 function, referred to as the binary logarithm and the inverse function of 2^N. The log2(n) is the power to which the number 2 must be raised to obtain the value n. Lets go back to our coin flip example with a fair coin. Suppose you flipped the coin once - the entropy would be 1 because there are two possible outcomes.

In [ ]:
np.log2(2) # log2(n) or 2^x = n? ...x = 1

##  If you flipped the coin twice you'd have 4 possible outcomes (00,10,01,11), or three times you'd have 8 possible outcomes:

In [ ]:
print(np.log2(4)) # or 2^x = n, x = 2
print(np.log2(8)) # or 2^x = n, x = 3

## To see the general shape of the function, plot out log2(x:y) to see the relationship between the number of possible outcomes and the entropy in bits

In [ ]:
# why am i going from 1,15 instead of our usual 0 based counting?
plt.plot(np.arange(1,15), np.log2(np.arange(1,15)), linewidth=3)
plt.ylim([-1,5])
plt.xticks(**fig_font)
plt.yticks(**fig_font)
plt.ylabel('Entropy (2^N = ?)', **fig_font)
plt.xlabel('# of possible outcomes', **fig_font)
plt.show()
# note that log2(0) == -inf, and that log2(1)==0

## Going back to the above notion that entropy is maximized when the outcome is completely uncertain (e.g. a fair coin), then we can start to develop an intuition about expressing entropy in terms of the probability that some event will occur, denoted P(X). 
* Lets take a more complex case - lets say that we have a slot machine that has two wheels on it and the first wheel can take one of N states and the second can take one of M states  when we pull the handle. Considering just the first wheel, the possible outcomes are {x1....xn}, and if each outcome is equally likely, then p(xi)= 1/n.  So, for example, if n = 16, then the total entropy of wheel 1 can be represented  by 4 bits of information. 

In [ ]:
np.log2(16)

## Now what about the second wheel? It can take on M states. So inutively the total number of possible outcomes for both wheels is N * M. The uncertainty of the outcome in this case is then:

In [ ]:
n = 16;
m = 16;
print(np.log2(n*m))

# which recall is equal to 
print(np.log2(n) + np.log2(m))

## Which gives us a nice way to account for the probability of outcomes across multiple variables via summation! Why is this computationally convienent, esp on digital computers?

## So in this case we need 8 bits of information to specify all possible outcomes. Now lets consider each possible outcome in isolation. what is the uncertainty associated with each event? its the probability of that event occuring, which is p(xi) = 1/n. So in terms of bits, we have :

In [ ]:
n = 16;
-np.log2(1/n)  #why negative when dealing with probabilities? log2(1)-log2(n)

## What we'd like though is a way to assess the average uncertainty of a particular outcome across all possible outcomes. How would you do that? 
* You'd take the uncertainty of each outcome (-log2(p(xi)) and weight it by the probability that the event will actually  occur, like this (where entropy is denoted, by convention, as H):

In [ ]:
# H = -sum_over_all_i( p(xi) * log2(p(xi)) )
n = 16;
H = 0;
for i in np.arange(n):
    H += -( (1/n) * np.log2(1/n) )

print(H)

## What does H, the average uncertaintly, equal when all events are equally likely? it equals the uncertaintly of each event, which it should. And note that this framework would let you compute the average uncertainty (entropy) across both slot machine wheels by just adding the entropies:

In [ ]:
n = 16 # number of possible outcomes for machine 1
m = 16 # number of possible outcomes for machine 2
H = 0
for i in np.arange(n):
    H += -( (1/n) * np.log2(1/n) )

for i in np.arange(m):
    H += -( (1/m) * np.log2(1/m) )

print(H)  

## What about when all events are not equally likely?

In [ ]:
# Remember that with 16 possible equally likely outcomes
# we should have an entropy of 4
n = 16

# pick some random values to use for p(x)
px = np.random.rand(n)
px /= np.sum(px) # convert the random numbers to probabilities

H = 0
for i in np.arange(n):
    H += -( px[i] * np.log2(px[i]) )

print(H)

plt.plot(np.arange(n),px)
plt.xlabel('Outcome', **fig_font)
plt.ylabel('P(outcome)', **fig_font)
plt.title('Prob of each of N outcomes', **fig_font)
plt.show()

<div class="alert alert-info">
Note that the entropy is lower because we've moved away from the point where everything is maximally unpredictable (i.e. a uniform distribution). This demonstrates a principle that relates probability distributions to entropy: uniform probability distributions have maximum entropy, and non-uniform distributions will generally have less entropy because some outcomes are more likely than others so you learn less by making a measurement.
</div> 

In [ ]:
# number of outcomes (e.g. heads or tails)
n = 2  

# vary the probability of heads
ph = np.arange(.001,.999,.001)  

# alloc to store entropy at each probability
H = np.zeros(len(ph))

for j in np.arange(len(ph)):
    px[0] = ph[j]    # p(heads)
    px[1] = 1-px[0]  # p(tails))
    
    # then compute entropy
    for i in np.arange(n):
        H[j] += -( px[i] * np.log2(px[i]) )
    

plt.plot(ph, H, 'k', linewidth=2)
plt.xlabel('Probability of heads (coin bias)', **fig_font)
plt.ylabel('entropy (bits)', **fig_font)
plt.axvline(.5, color='k', linewidth=2)
plt.show()
# entropy is maximized with maximum uncertainty and will max out at
# log2(n)

SUMMARY
* Entropy is a measure of uncertainty, and as uncertainty goes up (and is maximized when all outcomes are equally likely) then the information gained by making a measurement goes up. 
* So if you know the oucome in advance (e.g. a coin with two heads) then p(tails) = 0 and there is no uncertainty, entropy is 0, and there is no reduction in uncertainty to be gained by flipping the coin at all. 
* If you have a fair coin, then p(head)==p(tail) and entropy will be maximum and you will maximally reduce your uncertainty by making the measurement (in this case, you will fully disambiguate the outcome, gaining 1 bit of information where the total uncertainty is 1 bit).

## Mutual information (MI). 

* MI is a measure of how much knowing  about 1 variable tells you about the state of another variable. Putting aside entropy and measures of uncertainty/variance for a minute, here is the  intuition. 
    * Suppose you have two variables that are completely unrelated to each other: measuring one variable will tell you nothing about the state of the other variable. 
    * In contrast, if you have two variables that are perfectly correlated, then measuring one variable will tell you everything about the state of the other. 
    * In this special (unusual) case, the mutual information will be equal to the entropy of either variable alone (that is: the information gained by measuring one variable will be equal to the information gained by measuring both)
 
To put this back in terms of entropy: lets say we have two variables, X and Y. If we want to assess the MI between X and Y, then we need to know the following difference score:

(total entropy of X) -  (entropy of X given that we know Y). 

In other words, how much is uncertainty about X REDUCED when we measure Y? That is the MI between the two variables. And it leads to one common definition of MI:

MI = H(X) - H(X|Y)  

where H(X) is the entropy of X, and H(X|Y) is the conditional entropy of X given that we've measured Y - it is the average entropy of X across all values of Y

In [ ]:
# two discrete, uncorrelated arrays filled with 0's and 1's

N = 1000   # number of data points
x = np.round(np.random.rand(N))
y = np.round(np.random.rand(N))

px = np.zeros(2)
px[0] = np.sum(x)/N   # probability that x==1
px[1] = 1-px[0];      # prob that x==0

# do in one line instead of looping using the * operator
Hx = -sum( px * np.log2(px) )

# then compute average conditional entropy of x given y (Hxy).
# 1) Compute the entropy of X given each possible value of Y
# 2) Multiply H(X|Yi) with the probability of each Y (i.e. p(yi))
# 3) Sum H(X|Yi) over all i

# initialize Hxy
Hxy=0

# figure out the unique values in each vector (we know that its 0/1, but do this just for good practice)
uniquex = np.unique(x)
uniquey = np.unique(y)

# loop over unique elements of y, in this case 0,1
for i in np.arange(len(uniquey)): 
    
    # probability that y==y(i) (prob of each y)
    py = np.sum(y==uniquey[i]) / N

    # then loop over all possible x's to compute entropy of x at each y
    tmp=0
    for j in np.arange(len(uniquex)):
        px_y = np.sum((x==uniquex[j]) & (y==uniquey[i])) / np.sum(y==uniquey[i])    # e.g. prob x==1 when y==0
        tmp += (-( px_y * np.log2(px_y) ))                                                 # entropy      
        
    # then tally up entropy of x given each specific y multiplied by the probability of that y (py)
    Hxy += py*tmp

# then we have everything we need to compute MI, which in this case should
# be ~0 becuase the variables are completely independent!
MI = Hx - Hxy
print(MI)

## Now that we've walked all the way through one example, lets define a few functions to make things easier

In [ ]:
def entropy(x):
    """compute entropy of discrete array x

    Args:
        x (int): array of discrete values

    Returns:
        Hx (float): entropy of x

    """
    # figure out unique values of x - can be more than just 0s, 1s
    uniquex = np.unique(x)

    Hx = 0
    for i in np.arange(len(uniquex)):
        # probability that x==uniquex[i]
        px = np.sum(x==uniquex[i])/len(x)    

        # check for px==0 because log2(0) = -inf
        if px!=0:
            Hx += (-np.sum( px * np.log2(px) ))  
        else:
            print('px is zero for value ', i)
        
    return Hx

In [ ]:
def condEntropy(x,y):
    
    """
    conditional entropy, or the average entropy of x given each y, or Hxy
    1) For all Y {i=1:numel(X)}, compute the entropy of X given each Y
    2) Multiply H(X|Y==i) with the probability of each Y (i.e. pxi)
    3) Sum over all i

    Args:
        x (int): array of discrete values
        y (int): array of discrete values
        
    Returns:
        Hxy (float): average conditional entropy of x given y

    """

    Hxy=0
    uniquex = np.unique(x)
    uniquey = np.unique(y)

    # loop over unique elements of y
    for i in np.arange(len(uniquey)): 

        # probability that y==y(i) (prob of each y)
        py = np.sum(y==uniquey[i]) / N

        # then loop over all possible x's to compute entropy of x at each y
        tmp=0
        for j in np.arange(len(uniquex)):
            px_y = np.sum((x==uniquex[j]) & (y==uniquey[i])) / np.sum(y==uniquey[i])    # e.g. prob x==1 when y==0
            tmp += (-( px_y * np.log2(px_y) ))                                     # entropy      

        # then tally up entropy of x given each specific y multiplied by the probability of that y (py)
        Hxy += py*tmp

    return Hxy

## Lets give the functions a try

In [ ]:
N = 1000   # number of data points
x = np.round(np.random.rand(N))
y = np.round(np.random.rand(N))
Hx = entropy(x=x)
Hxy = condEntropy(x=x,y=y)
print('MI is: ', Hx-Hxy)

## Now lets try a more complicated (and useful) example where we have one discrete variable and one continuous variable
* This often comes up in experiments where you have a set of discrete stimuli and then a continuous output measure (e.g. LFP power, spike rate over a time window, etc)
* Logic here is the same as the case for two discrete variables, but generating the PDF for the continuous variable is trickier and there are many (many many) ways of doing it. 
    * For comparison, with our coin tosses, you can get the PDF by sum(heads)/total_tosses...
    * But for a continuous metric where you have all unique values, this doesn't work (i.e. if each value happens once and only once then how do you compute the likelihood of observing that value?
    * To solve this problem, we need to generate an estimate of the data generating PDF based on our samples of the continuous data
    * Common approaches are to build histograms of the data to approximate the PDF, or to use an approach like Kernel Density Estimation to approximate the PDF 
* We'll use the KDE implementation in sklearn 
* We'll also turn the MI formula around a bit more (its symetric so this is ok)
    * Instead of MI = Hx - Hxy, we'll use MI = Hy - Hyx
    
[histograms vs KDE - good read!](https://mglerner.github.io/posts/histograms-and-kernel-density-estimation-kde-2.html?p=28)

[And a TMI read for muliple KDE implementations](https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/)

## Import KDE functionality from sklearn

In [ ]:
from sklearn.neighbors import KernelDensity

## Set up our variables - one discrete and one continuous
* lets assume that our data came from an experiment where we show 4 different stimuli (e.g. motion directions) and then record spike rate as a continuous metric (e.g. spike rate smoothed with a 5ms Gaussian window). 

In [ ]:
# number of trials in our experiment
N = 1000

# Generate a list of stims
num_stims = 4
stims = np.repeat(np.arange(4), N/num_stims) # div by num_stims to keep output length constant

# Now generate our list of continuous spike rates in response to each stimulus
# Set it up so that there is a differential response to each of the 4 stims (i.e. MI>0)
scale_factor = 2 # how much response to each stim differs
resp=[]
for i in np.arange(num_stims):
    resp = np.hstack((resp, np.random.randn(int(N/num_stims))+(i*scale_factor)))
    
plt.plot(np.arange(N), resp, 'k', linewidth=2)
plt.xlabel('Trial Number', **fig_font)
plt.ylabel('Resp(Hz)', **fig_font)
plt.show()

# also show as a histogram to get a feel for the distribution
num_bins = 10 # change this! you'll get a feel for why using histograms for estimating PDFs is tricky
plt.hist(resp, bins=num_bins)
plt.xlabel('Resp(Hz)', **fig_font)
plt.ylabel('Trial count', **fig_font)
plt.show()

## Now use sklearn KDE algorithm to estimate a PDF of our continuous response

In [ ]:
# first define a set of points over which to evaluate the PDF

# figure out min and max of our data - good start for picking a range (use ceil and floor...)
min_resp = np.floor(np.min(resp))
max_resp = np.ceil(np.max(resp))

# then define the number of points that we want to eval the function over
# I'll choose N here 
num_points = N

# then define the x_range
x_range = np.linspace(min_resp, max_resp, num_points)

# KDE function expects a 2d array, so expand x to have a singular second dim
if len(x_range.shape):
    x_range = x_range[:, np.newaxis]
    
print(x_range.shape)

In [ ]:
# then lets call our KDE function and we'll just pick an arbitrary bandwidth for the kernel
bw = .2

# make our data a N,1 matrix for input to KDE function
if len(resp.shape)==1:
    resp = resp[:,np.newaxis]

kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(resp)
log_dens = kde.score_samples(x_range)

# plot
plt.fill(x_range[:, 0], np.exp(log_dens), alpha=0.5)
plt.show()

num_bins=30
plt.hist(resp, bins = num_bins, alpha=0.5, normed=1)
plt.show()

## Notice that I just selected an arbitrary kernel bandwidth to estimate the PDF
* First go back and play with it a bit in the above cell to see what happens as you make it smaller/bigger
* We can then make this choice in a more principled manner by using cross-validation where part of the data is held out, a kernel fit is estimated and applied to the remaining data, and this process is iterated over a bunch of kernels until the one with the best cross-validated maximum likelihood is found. 

In [ ]:
from sklearn.grid_search import GridSearchCV

# number of cross-validation folds
cv_folds = 10
min_bw = 0.1
max_bw = 1.0
bw_steps = 50
grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.linspace(min_bw, max_bw, bw_steps)},
                    cv=cv_folds) 
grid.fit(resp)

best_bandwidth = grid.best_params_

print(best_bandwidth)

In [ ]:
# then lets call our KDE function with the best bandwidth kernel
bw = best_bandwidth['bandwidth']

# make our data a N,1 matrix for input to KDE function
if len(resp.shape)==1:
    resp = resp[:,np.newaxis]

kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(resp)
log_dens = kde.score_samples(x_range)

# save out p(y) function
py = np.exp(log_dens)

# normalize to unit area
py /= np.sum(py)

# plot!
plt.fill(x_range[:, 0], py, alpha=0.5)
plt.show()

## First compute the entropy of the continuous variable 

In [ ]:
Hy = -sum(py * np.log2(py))  
print(Hy)

## Now that we know how to compute an estimate of the PDF of our continuous variable we can compute MI by figuring out Hyx, or the average entropy of y at each given x

In [ ]:
#MI = Hy - Hyx

# find num unique stims
uniquex = np.unique(stims)

# do a cheat here - we know that x has an equal number of 0,1,2,3... in it, so we can figure
# out the probability of any given value by just computing it for one entry
px = N/num_stims/N

# then loop over all possible x's to compute entropy of y given each x (Hyx)
Hyx=0
for j in np.arange(len(uniquex)):
    
    # grab data from all trials where stims == j
    y_x = resp[stims==j]
    
    # do KDE on this subject of data conditioned on x
    # note: could cross-val to determine bw here as well
    
    # compute KDE
    kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(y_x)
    log_dens = kde.score_samples(x_range)

    # save out kde solution
    tmp_y_x = np.exp(log_dens)

    # normalize to unit area
    tmp_y_x /= np.sum(tmp_y_x)
    
    # then tally up entropy of y given each specific x multiplied by the probability of that x (px)
    Hyx += px*(-sum( tmp_y_x * np.log2(tmp_y_x) ))                                    


In [ ]:
MI = Hy - Hyx
print(MI)